# Azure PowerShell Visualization demo  <img src="https://raw.githubusercontent.com/PowerShell/PowerShell/master/assets/Powershell_black_64.png" align="right"/>
> A Notebook mixing Azure PowerShell and Plotly

## Prerequisites

### Install required modules

You'll need to install `Az.Compute`, `Az.Resources` and `Az.Monitor` for this Notebook.

In [ ]:
Install-Module Az.Compute,Az.Resources,Az.Monitor -Force

### Authenticate and set context

First connect to your Azure account.

In [ ]:
Connect-AzAccount

**(Optional)** If your account contains more than one active subscription the first one will be selected for further use. To select another subscription, use Set-AzContext.

In [ ]:
Set-AzContext -Subscription "My Subscription"

### (Optional) Populate Azure with test VMs

Here's a script that will create a number of VMs. Feel free to change the value at the top.

> NOTE: This can take a LONG time.

In [ ]:
# IMPORTANT VARIABLES
$RESOURCE_GROUP_NAME = 'VMVisDemo'
$LOCATION = 'East US 2'
$NUM_OF_VMs = 10
$USER_CREDENTIAL = Get-Credential

In [ ]:
Write-Host "Creting resource group."
New-AzResourceGroup -Name $RESOURCE_GROUP_NAME -Location $LOCATION | Out-Null

Write-Host "Creting VMs."
$jobs = 1..$NUM_OF_VMs | ForEach-Object {
    $splat = @{
        Image = "UbuntuLTS"
        Location = $LOCATION
        Name = "MyVM-$_"
        ResourceGroupName = $RESOURCE_GROUP_NAME
        Credential = $USER_CREDENTIAL
        AsJob = $true
    }
    New-AzVM @splat
}

# Wait for them to all be created
Wait-Job $jobs | Out-Null
"Done!"

Now we need to randomly stop a number of them so that the graph below has some variance.

In [ ]:
$numOfVMsToStop = Get-Random -Minimum 2 -Maximum $NUM_OF_VMs
Write-Host "Randomly stoping $numOfVMsToStop VMs."

$vms = Get-AzVM -ResourceGroupName $RESOURCE_GROUP_NAME
$jobs = for ($i = 0; $i -lt $numOfVMsToStop; $i++) {
    $vms | Get-Random | Stop-AzVM -Force -AsJob
}

Wait-Job $jobs | Out-Null

Write-Host "Done!"

**Cleanup** - If you wanna clean up these test VMs. Run this:

> NOTE: This can take a LONG time.

In [ ]:
Write-Host "Deleting VMs."
$jobs = Get-AzVM -ResourceGroupName $RESOURCE_GROUP_NAME | Remove-AzVM -AsJob -Force
Wait-Job $jobs | Out-Null

Write-Host "Deleting resource group."
Remove-AzResourceGroup -ResourceGroupName $RESOURCE_GROUP_NAME -Force | Out-Null

Write-Host "Done!"

## Demo 1 - Pie Chart of VM PowerState

We'll be fetching the "PowerState" of each of our VMs in Azure and plotting it on a pie chart.

First, let's get the data:

In [ ]:
$vms = Get-AzVM -Status
$data = $vms.PowerState

$groupedData = $data | Group-Object
$groupedData

Now we can render that data into a Pie graph and plot it in a chart:

In [ ]:
$trace = [Graph.Pie]@{
    name = "VM PowerState"
    labels = $groupedData.Name
    values = [int[]]($groupedData | % Count)
}

New-PlotlyChart -Title "VM Status" -Trace $trace | Out-Display

## Demo 2 - Plotting Azure Monitor logs

We'll be fetching the Azure Monitor logs and plotting.

First, let's get the data:

In [ ]:
$logs = Get-AzLog -MaxRecord 1000

### Plot events per resource group

Now let's plot the data:

In [ ]:
$entryByRGN = [Graph.Histogram]@{
    name = "ResourceGroup"
    x = $logs.ResourceGroupName
}

$entryByRGN | New-PlotlyChart -Title "Events per ResourceGroup" | Out-Display

### Plot events over time

We can continue to reuse the `$logs` variable and plot something else:

In [ ]:
$eventsOverTime = [Graph.Histogram]@{
    name = "Events"
    x = $logs.EventTimestamp
    marker = [Graph.Marker]@{
        color = "coral"
    }
}

$eventsOverTime | New-PlotlyChart -Title "Events over time" | Out-Display